
# INSTALLATION

In [1]:
!pip install aif360
!pip install fairlearn

     |████████████████████████████████| 175 kB 29.6 MB/s 
     |████████████████████████████████| 356 kB 52.9 MB/s 
  Created wheel for memory-profiler: filename=memory_profiler-0.58.0-py3-none-any.whl size=30190 sha256=deed5b1a0f448dd4a0bb54f3631085fe8b2d595ba56d9ac1bca6c7321b38b512
  Stored in directory: /root/.cache/pip/wheels/56/19/d5/8cad06661aec65a04a0d6785b1a5ad035cb645b1772a4a0882
  Created wheel for shap: filename=shap-0.39.0-cp37-cp37m-linux_x86_64.whl size=491653 sha256=edb5cc248a8337bd0b954cd2735657de2f85f9cf9f1bf546dcef31976de23bbb
  Stored in directory: /root/.cache/pip/wheels/ca/25/8f/6ae5df62c32651cd719e972e738a8aaa4a87414c4d2b14c9c0
Successfully built memory-profiler shap
     |████████████████████████████████| 177 kB 27.0 MB/s 


In [2]:
!apt-get install -jre
!java -version

E: Command line option 'j' [from -jre] is not understood in combination with the other options.
openjdk version "11.0.11" 2021-04-20
OpenJDK Runtime Environment (build 11.0.11+9-Ubuntu-0ubuntu2.18.04)
OpenJDK 64-Bit Server VM (build 11.0.11+9-Ubuntu-0ubuntu2.18.04, mixed mode, sharing)


In [3]:
!pip install h2o

     |████████████████████████████████| 168.4 MB 17 kB/s 
  Created wheel for h2o: filename=h2o-3.32.1.6-py2.py3-none-any.whl size=168439194 sha256=b36dfb2acf145862303b77bb6e05ac011a80c8b90820d4b3499dd725f6f610bc
  Stored in directory: /root/.cache/pip/wheels/ee/0f/51/849ba221c4c1b11a04efb4a3427dc9cb1c4dcde218c6c98b13
Successfully built h2o


In [4]:
!pip install xlsxwriter

     |████████████████████████████████| 148 kB 36.6 MB/s 


#IMPORTS

In [12]:
import numpy as np
from mlxtend.feature_selection import  ExhaustiveFeatureSelector
from xgboost import  XGBClassifier
# import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import openpyxl
import xlsxwriter
from openpyxl import  load_workbook

import shap
#suppress setwith copy warning
pd.set_option('mode.chained_assignment',None)
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest, SelectFwe, SelectPercentile,SelectFdr, SelectFpr, SelectFromModel
from sklearn.feature_selection import chi2, mutual_info_classif
# from skfeature.function.similarity_based import fisher_score
import aif360
import matplotlib.pyplot as plt
from aif360.metrics.classification_metric import ClassificationMetric
from aif360.algorithms.postprocessing import EqOddsPostprocessing, RejectOptionClassification,CalibratedEqOddsPostprocessing

from aif360.metrics import BinaryLabelDatasetMetric
from aif360.datasets import StandardDataset , BinaryLabelDataset
from sklearn.preprocessing import MinMaxScaler 
MM= MinMaxScaler()
import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.glm import H2OGeneralizedLinearEstimator

import sys
sys.path.append("../")
import os


In [13]:
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 . connected.


H2O_cluster_uptime:,35 mins 47 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.32.1.6
H2O_cluster_version_age:,4 days
H2O_cluster_name:,H2O_from_python_unknownUser_tw2yxi
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


#**************************LOADING DATASET*******************************

In [14]:
from google.colab import drive 
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [15]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'], axis=1)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'], axis=1)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  
  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "first_pf"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()
  aml = H2OAutoML(max_models=10, nfolds=10, include_algos=['GBM'] , stopping_metric='AUTO') #verbosity='info',,'GBM', 'DRF'
  aml.train(x=x, y=y, training_frame=Train)
  best_model= aml.leader
  # a.model_performance()
  #**********************REPLACE LABELS OF DUPLICATED TRAIN AND  TEST SET WITH 0.5 THRESHOLDED PREDICT PROBA****************************
  #predicted proba for train labels
  gbm_Predictions_train= best_model.predict(Train)
  gbm_Predictions_train= gbm_Predictions_train.as_data_frame()
  train_label= (gbm_Predictions_train.p1>0.5).astype(int)
  predicted_df_train= train.copy()
  predicted_df_train['first_pf']= train_label

  #predicted proba for test labels
  gbm_Predictions_test= best_model.predict(Test)
  gbm_Predictions_test= gbm_Predictions_test.as_data_frame()
  test_label= (gbm_Predictions_test.p1>0.5).astype(int)
  predicted_df_test= test.copy()
  predicted_df_test['first_pf']= test_label


  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************
  '''the EO's fit method takes as input the original data (ground truth) and the predicted dataset with the predicted dataset having 
  same features but different labels (label for predicted dataset is the thresholded predict proba). as TPR and FPR requires \hat{Y} and Y 
  Converting the  two required dataset to binary format as accepted by EO.
  ''' 


  #Transforming  the Ground truth Train Test to BLD
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_Train= Train(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #**************************************Predicted Train Test BLD*****************************************
  class PreTrain(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTrain, self).__init__(df=predicted_df_train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_PredictedTrain= PreTrain(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class PreTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTest, self).__init__(df=predicted_df_test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredictedTest= PreTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #****************************************Equal Opportunity Instance*******************************
  Equalizer= CalibratedEqOddsPostprocessing(unprivileged_groups= disadvantagedGroup , privileged_groups= advantagedGroup )
  #uses ground truth and unfair predictions of classifier to determine probabilities with which unfair output labels are changed to satisfy EO
  Equalizer.fit(BLD_Train,BLD_PredictedTrain )
  #predicting the new labels assigned by the EO engine
  BLD_PredictedTest= Equalizer.predict(BLD_PredictedTest)

  # ********************COMPUTE DISCRIMINATION*****************************

  excelBook= load_workbook('/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_gbm.xlsx')
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredictedTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_gbm.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.901124111034641
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
AutoML progress: |████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.8942175309775127
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████

#LOGISTIC REGRESSION

In [ ]:
for i in range(1,51,1):

  train_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Train'
  train_path= os.path.join(train_url ,("Train"+ str(i)+ ".csv"))
  train= pd.read_csv(train_path).drop(['region_first'], axis=1)

  test_url=r'/content/gdrive/MyDrive/Datasets/SurveyData/DATASET/Law/Test'
  test_path= os.path.join(test_url ,("Test"+ str(i)+ ".csv"))
  test= pd.read_csv(test_path).drop(['region_first'], axis=1)

  # normalization of train and test sets
  Fitter= MM.fit(train)
  transformed_train=Fitter.transform(train)
  train=pd.DataFrame(transformed_train, columns= train.columns)

  #test normalization
  transformed_test=Fitter.transform(test)
  test=pd.DataFrame(transformed_test, columns= test.columns)

  # ********************SETTING TO H20 FRAME AND MODEL TRAINING*******************************
  x = list(train.columns)
  y = "first_pf"
  x.remove(y)
  Train=h2o.H2OFrame(train)
  Test= h2o.H2OFrame(test)
  Train[y] = Train[y].asfactor()
  Test[y] = Test[y].asfactor()

  LogReg = H2OGeneralizedLinearEstimator(family= "binomial", lambda_ = 0)
  LogReg.train(x=x, y=y, training_frame=Train)

  #**********************REPLACE LABELS OF DUPLICATED TRAIN AND  TEST SET WITH 0.5 THRESHOLDED PREDICT PROBA****************************
  #predicted proba for train labels
  lr_Predictions_train= LogReg.predict(Train)
  lr_Predictions_train= lr_Predictions_train.as_data_frame()
  train_label= (lr_Predictions_train.p1>0.5).astype(int)
  predicted_df_train= train.copy()
  predicted_df_train['first_pf']= train_label

  #predicted proba for test labels
  lr_Predictions_test= LogReg.predict(Test)
  lr_Predictions_test= lr_Predictions_test.as_data_frame()
  test_label= (lr_Predictions_test.p1>0.5).astype(int)
  predicted_df_test= test.copy()
  predicted_df_test['first_pf']= test_label


  # *************CHECKING FAIRNESS IN DATASET**************************
  ## ****************CONVERTING TO BLD FORMAT******************************
  '''the EO's fit method takes as input the original data (ground truth) and the predicted dataset with the predicted dataset having 
  same features but different labels (label for predicted dataset is the thresholded predict proba). as TPR and FPR requires \hat{Y} and Y 
  Converting the  two required dataset to binary format as accepted by EO.
  ''' 


  #Transforming  the Ground truth Train Test to BLD
  advantagedGroup= [{'race':1}]
  disadvantagedGroup= [{'race':0}]

  class Train(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Train, self).__init__(df=train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_Train= Train(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class Test(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(Test, self).__init__(df=test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_Test= Test(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #**************************************Predicted Train Test BLD*****************************************
  class PreTrain(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                   favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTrain, self).__init__(df=predicted_df_train  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
             )

  BLD_PredictedTrain= PreTrain(protected_attribute_names= ['race'],
                         privileged_classes= [[1]])

  class PreTest(StandardDataset):
      def __init__(self,label_name= 'first_pf',
                  favorable_classes= [1],protected_attribute_names=['race'],   privileged_classes=[[1]], ):


          super(PreTest, self).__init__(df=predicted_df_test  , label_name=label_name ,
              favorable_classes=favorable_classes , protected_attribute_names=protected_attribute_names ,
              privileged_classes=privileged_classes ,
            )

  BLD_PredictedTest= PreTest(protected_attribute_names= ['race'],
                        privileged_classes= [[1]])
  
  #****************************************Equal Opportunity Instance*******************************
  Equalizer= CalibratedEqOddsPostprocessing(unprivileged_groups= disadvantagedGroup , privileged_groups= advantagedGroup )
  #uses ground truth and unfair predictions of classifier to determine probabilities with which unfair output labels are changed to satisfy EO
  Equalizer.fit(BLD_Train,BLD_PredictedTrain )
  #predicting the new labels based on the fitted EO engine
  BLD_PredictedTest= Equalizer.predict(BLD_PredictedTest)

  # ***************************COMPUTE DISCRIMINATION********************************

  excelBook= load_workbook("/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_LogReg.xlsx")
  Law= excelBook['Law']
  data= Law.values

  # Get columns
  columns = next(data)[0:]
  10# Create a DataFrame based on the second and subsequent lines of data
  OldDF = pd.DataFrame(data, columns=columns)

  ClassifierBias = ClassificationMetric( BLD_Test,BLD_PredictedTest    , unprivileged_groups= disadvantagedGroup, privileged_groups= advantagedGroup)
  Accuracy= ClassifierBias.accuracy()
  TPR= ClassifierBias.true_positive_rate()
  TNR= ClassifierBias.true_negative_rate()
  NPV= ClassifierBias.negative_predictive_value()
  PPV= ClassifierBias.positive_predictive_value()
  SP=ClassifierBias .statistical_parity_difference() 
  IF=ClassifierBias.consistency()
  DI=ClassifierBias.disparate_impact()
  EOP=ClassifierBias.true_positive_rate_difference()
  EO=ClassifierBias.average_odds_difference()
  FDR= ClassifierBias.false_discovery_rate(privileged=False)- ClassifierBias.false_discovery_rate(privileged=True)
  NPV_diff=ClassifierBias.negative_predictive_value(privileged=False)-ClassifierBias.negative_predictive_value(privileged=True)
  FOR=ClassifierBias.false_omission_rate(privileged=False)-ClassifierBias.false_omission_rate(privileged=True)
  PPV_diff=ClassifierBias.positive_predictive_value(privileged=False) -ClassifierBias.positive_predictive_value(privileged=True)
  BGE = ClassifierBias.between_group_generalized_entropy_index()
  WGE = ClassifierBias.generalized_entropy_index()-ClassifierBias.between_group_generalized_entropy_index()
  BGTI = ClassifierBias.between_group_theil_index()
  WGTI = ClassifierBias.theil_index() -ClassifierBias.between_group_theil_index()
  EDF= ClassifierBias.differential_fairness_bias_amplification()

  newdf= pd.DataFrame(index = [0], data= { 'ACCURACY': Accuracy,'TPR': TPR, 'PPV':PPV, 'TNR':TNR,'NPV':NPV,'SP':SP,'CONSISTENCY':IF,'DI':DI,'EOP':EOP,'EO':EO,'FDR':FDR,'NPV_diff':NPV_diff, 
                                          'FOR':FOR,'PPV_diff':PPV_diff,'BGEI':BGE,'WGEI':WGE,'BGTI':BGTI,'WGTI':WGTI,'EDF':EDF})
  newdf=pd.concat([OldDF,newdf])

  pathway= r"/content/gdrive/MyDrive/Datasets/SurveyData/RESULTS/CalEqualOdds/CalEO_LogReg.xlsx"

  with pd.ExcelWriter(pathway, engine='openpyxl') as writer:
    #load workbook base as for writer
    writer.book= excelBook
    writer.sheets=dict((ws.title, ws) for ws in excelBook.worksheets)
    newdf.to_excel(writer, sheet_name='Law', index=False)
    # newdf.to_excel(writer, sheet_name='Adult', index=False)

  print('Accuracy', Accuracy)

Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.9025005735260381
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%
glm Model Build progress: |███████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
glm prediction progress: |████████████████████████████████████████████████| 100%
Accuracy 0.8921523634694815
Parse progress: |█████████████████████████████████████████████████████████| 100%
Parse progress: |████████████████████████████████████